In [4]:
import pandas as pd
import numpy as np

# 首先设置matplotlib为非交互模式，确保不显示任何图表
import matplotlib
matplotlib.use('Agg')  # 使用非GUI后端，确保不显示图表
import matplotlib.pyplot as plt
plt.ioff()  # 关闭交互模式

import seaborn as sns
from sklearn.inspection import partial_dependence, PartialDependenceDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.ensemble import IsolationForest
import pickle
import joblib
import os
from itertools import combinations
import warnings
from matplotlib import rcParams
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from scipy.interpolate import UnivariateSpline
from matplotlib.colors import LinearSegmentedColormap

warnings.filterwarnings('ignore')

# 设置matplotlib为非交互模式，不显示图表
plt.ioff()  # 关闭交互模式
matplotlib.use('Agg')  # 使用非GUI后端

# 设置中文字体和绘图风格
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['figure.max_open_warning'] = 0  # 禁用图形数量警告
plt.rcParams['interactive'] = False  # 禁用交互模式
sns.set_style("whitegrid")

# 配置参数 - 修改为GBDT模型专用配置
CONFIG = {
    # 修改：指向GBDT模型的输出目录
    'gbdt_model_dir': "D:\\博一下\\manuscript5-变Si组\\最优模型GBDT专用训练",
    'data_file': "D:\\博一下\\Nb-Si数据库8.28特征值.xlsx",
    'pdp_output_dir': "D:\\博一下\\manuscript5-变Si组\\GBDT模型PDP分析结果",
    
    # GBDT模型相关信息
    'selected_features': [
        'mean_C4 enthalpy melting', 
        'solubility_limit_factor', 
        'mean_S10 Lattice Constants a', 
        'Λ', 
        'Ω'
    ],
    'target_col': 'KQ',
    'model_timestamp': '1756353666',  # 修改：使用你的模型时间戳
    'random_state': 2023,
    
    'grid_resolution': 200,  # 从100改为200，曲线更光滑
    'percentiles': (0.01, 0.99),  # PDP计算的百分位数范围
    'figure_size': (12, 8),  # 图形尺寸
    'dpi': 300,  # 图像分辨率
    
    # 学术标准的Bootstrap设置
    'bootstrap_samples': 1000,  # 提升到1000次（符合学术标准）
    'bootstrap_samples_small': 2000,  # 小数据集使用更多次数
    'bootstrap_samples_large': 500,   # 大数据集可以减少次数
    
    'confidence_level': 0.95,   # 置信水平
    'small_dataset_threshold': 1000,   # 小数据集阈值
    'large_dataset_threshold': 10000,  # 大数据集阈值
    
    # 新增：柱状图配置
    'histogram_bins': 50,  # 柱状图箱数
    
    # 交互分析配置
    'interaction_2d_resolution': 50,  # 2D交互分析网格分辨率
    'interaction_3d_resolution': 30,  # 3D交互分析网格分辨率
    'max_3d_interactions': 10,  # 最大3D交互分析数量
    
    # 增强：异常值检测配置
    'outlier_detection': {
        'enable': True,  # 是否启用异常值检测
        'auto_clean': True,  # 是否自动清洗异常值
        'contamination': 0.05,  # 异常值比例假设 (5%)
        'min_votes': 2,  # 异常值检测最少投票数
        'z_score_threshold': 3.0,  # Z-Score阈值
        'iqr_multiplier': 1.5,  # IQR倍数
        'target_features': ['Ω'],  # 重点检测的特征（可以为空表示检测所有特征）
        'save_outlier_reports': True,  # 是否保存异常值检测报告
        'comparison_analysis': True,  # 是否进行清洗前后对比分析
        'unified_cleaning': True,  # 是否统一清洗（确保所有分析使用相同的清洗后数据）
        'multi_feature_detection': True,  # 是否启用多特征联合异常值检测
    }
}

def get_adaptive_config(n_samples):
    """根据数据集大小自适应调整参数"""
    if n_samples <= CONFIG['small_dataset_threshold']:
        return {
            'bootstrap_samples': CONFIG['bootstrap_samples_small'],
            'description': '小数据集-高精度Bootstrap'
        }
    elif n_samples <= CONFIG['large_dataset_threshold']:
        return {
            'bootstrap_samples': CONFIG['bootstrap_samples'],
            'description': '中等数据集-标准Bootstrap'
        }
    else:
        return {
            'bootstrap_samples': CONFIG['bootstrap_samples_large'],
            'description': '大数据集-高效Bootstrap'
        }

def cleanup_matplotlib():
    """清理matplotlib资源"""
    plt.close('all')
    import gc
    gc.collect()

def calculate_histogram_data(feature_data, bins=None):
    """计算特征数据的直方图分布数据"""
    if bins is None:
        bins = CONFIG['histogram_bins']
    
    # 计算直方图数据
    counts, bin_edges = np.histogram(feature_data, bins=bins)
    
    # 计算箱中心点
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # 计算密度（归一化频率）
    bin_width = bin_edges[1] - bin_edges[0]
    densities = counts / (len(feature_data) * bin_width)
    
    # 计算相对频率（百分比）
    relative_frequencies = counts / len(feature_data) * 100
    
    return {
        'bin_centers': bin_centers,
        'bin_edges': bin_edges,
        'counts': counts,
        'densities': densities,
        'relative_frequencies': relative_frequencies,
        'bin_width': bin_width,
        'total_samples': len(feature_data)
    }

class UnifiedOutlierDetector:
    """统一异常值检测器类 - 用于全局数据清洗"""
    
    def __init__(self, config):
        self.config = config
        self.global_outlier_mask = None
        self.global_outlier_report = {}
        
    def detect_global_outliers(self, X_data, y_data, features, model=None):
        """全局异常值检测 - 统一处理所有特征"""
        
        print(f"\n🔍 开始全局异常值检测（统一清洗模式）...")
        print(f"  数据集大小: {X_data.shape[0]} 样本 × {X_data.shape[1]} 特征")
        
        n_samples, n_features = X_data.shape
        all_outlier_methods = {}
        feature_outlier_reports = {}
        
        # 1. 逐特征异常值检测
        print(f"  步骤 1/3: 逐特征异常值检测...")
        for i, feature in enumerate(features):
            feature_data = X_data[:, i]
            
            # 检查是否为重点检测特征
            target_features = self.config.get('target_features', [])
            should_detect = (not target_features) or (feature in target_features)
            
            if should_detect:
                print(f"    🎯 检测特征: {feature}")
                outlier_methods = self._detect_single_feature_outliers(feature_data, feature)
                
                # 添加PDP跳跃检测（如果提供了模型）
                if model is not None:
                    jump_outliers = self._detect_pdp_jump_outliers(model, X_data, i, feature_data, feature)
                    if np.sum(jump_outliers) > 0:
                        outlier_methods['pdp_jump'] = jump_outliers
                
                # 保存单特征检测结果
                for method, mask in outlier_methods.items():
                    method_key = f"{feature}_{method}"
                    all_outlier_methods[method_key] = mask
                
                feature_outlier_reports[feature] = outlier_methods
            else:
                print(f"    ⏭️ 跳过特征: {feature} (不在重点检测列表中)")
        
        # 2. 多特征联合异常值检测
        if self.config.get('multi_feature_detection', False):
            print(f"  步骤 2/3: 多特征联合异常值检测...")
            multi_feature_outliers = self._detect_multi_feature_outliers(X_data, features)
            for method, mask in multi_feature_outliers.items():
                all_outlier_methods[f"multi_{method}"] = mask
        else:
            print(f"  步骤 2/3: 跳过多特征检测 (已禁用)")
        
        # 3. 创建全局异常值掩码
        print(f"  步骤 3/3: 创建全局异常值掩码...")
        if all_outlier_methods:
            # 计算每个样本被标记为异常值的次数（跨所有方法和特征）
            method_names = list(all_outlier_methods.keys())
            vote_matrix = np.column_stack([all_outlier_methods[method] for method in method_names])
            vote_counts = np.sum(vote_matrix, axis=1)
            
            # 使用投票机制确定最终异常值
            self.global_outlier_mask = vote_counts >= self.config['min_votes']
            outlier_count = np.sum(self.global_outlier_mask)
            
            print(f"    ✅ 全局异常值检测完成:")
            print(f"      - 检测方法总数: {len(all_outlier_methods)}")
            print(f"      - 异常值样本数: {outlier_count}/{n_samples} ({outlier_count/n_samples*100:.2f}%)")
            print(f"      - 投票阈值: {self.config['min_votes']}")
            
            # 保存全局异常值报告
            self.global_outlier_report = {
                'all_methods': all_outlier_methods,
                'feature_reports': feature_outlier_reports,
                'vote_counts': vote_counts,
                'vote_matrix': vote_matrix,
                'method_names': method_names,
                'outlier_mask': self.global_outlier_mask,
                'outlier_count': outlier_count,
                'outlier_percentage': outlier_count/n_samples*100
            }
            
        else:
            print(f"    ⚠️ 没有检测到任何异常值方法结果")
            self.global_outlier_mask = np.zeros(n_samples, dtype=bool)
            self.global_outlier_report = {}
        
        return self.global_outlier_mask, self.global_outlier_report
    
    def _detect_single_feature_outliers(self, feature_data, feature_name):
        """单特征异常值检测"""
        
        outlier_methods = {}
        n_samples = len(feature_data)
        
        # 方法1: Z-Score方法
        z_scores = np.abs(stats.zscore(feature_data))
        outlier_methods['z_score'] = z_scores > self.config['z_score_threshold']
        
        # 方法2: 改进的IQR方法
        Q1 = np.percentile(feature_data, 25)
        Q3 = np.percentile(feature_data, 75)
        IQR = Q3 - Q1
        iqr_lower = Q1 - self.config['iqr_multiplier'] * IQR
        iqr_upper = Q3 + self.config['iqr_multiplier'] * IQR
        outlier_methods['iqr'] = (feature_data < iqr_lower) | (feature_data > iqr_upper)
        
        # 方法3: 基于分位数的方法
        p1, p99 = np.percentile(feature_data, [1, 99])
        p0_5, p99_5 = np.percentile(feature_data, [0.5, 99.5])
        outlier_methods['percentile_1_99'] = (feature_data < p1) | (feature_data > p99)
        outlier_methods['percentile_0.5_99.5'] = (feature_data < p0_5) | (feature_data > p99_5)
        
        # 方法4: Isolation Forest
        try:
            iso_forest = IsolationForest(
                contamination=self.config['contamination'], 
                random_state=42
            )
            outlier_methods['isolation_forest'] = iso_forest.fit_predict(feature_data.reshape(-1, 1)) == -1
        except:
            outlier_methods['isolation_forest'] = np.zeros(n_samples, dtype=bool)
        
        # 统计各方法结果
        for method, mask in outlier_methods.items():
            count = np.sum(mask)
            percentage = count / n_samples * 100
            print(f"      {method}: {count} 个异常值 ({percentage:.2f}%)")
        
        return outlier_methods
    
    def _detect_multi_feature_outliers(self, X_data, features):
        """多特征联合异常值检测"""
        
        multi_outlier_methods = {}
        n_samples = X_data.shape[0]
        
        try:
            # 多维Isolation Forest
            iso_forest_multi = IsolationForest(
                contamination=self.config['contamination'],
                random_state=42
            )
            multi_outlier_methods['isolation_forest_multi'] = iso_forest_multi.fit_predict(X_data) == -1
            
            # 马氏距离异常值检测
            try:
                from scipy.spatial.distance import mahalanobis
                mean = np.mean(X_data, axis=0)
                cov = np.cov(X_data.T)
                cov_inv = np.linalg.pinv(cov)  # 使用伪逆处理奇异矩阵
                
                mahal_distances = []
                for i in range(n_samples):
                    try:
                        distance = mahalanobis(X_data[i], mean, cov_inv)
                        mahal_distances.append(distance)
                    except:
                        mahal_distances.append(np.nan)
                
                mahal_distances = np.array(mahal_distances)
                # 移除NaN值
                valid_distances = mahal_distances[~np.isnan(mahal_distances)]
                if len(valid_distances) > 0:
                    threshold = np.percentile(valid_distances, 95)  # 95%分位数
                    multi_outlier_methods['mahalanobis'] = (mahal_distances > threshold) & (~np.isnan(mahal_distances))
                
            except Exception as e:
                print(f"      ⚠️ 马氏距离计算失败: {str(e)}")
        
        except Exception as e:
            print(f"      ⚠️ 多特征异常值检测失败: {str(e)}")
        
        # 统计多特征检测结果
        for method, mask in multi_outlier_methods.items():
            count = np.sum(mask)
            percentage = count / n_samples * 100
            print(f"      {method}: {count} 个异常值 ({percentage:.2f}%)")
        
        return multi_outlier_methods
    
    def _detect_pdp_jump_outliers(self, model, X_data, feature_idx, feature_data, feature_name):
        """检测导致PDP异常跳跃的数据点"""
        
        print(f"      🔍 PDP跳跃检测: {feature_name}")
        
        try:
            # 计算粗略PDP来找到异常跳跃点
            temp_grid = np.linspace(np.min(feature_data), np.max(feature_data), 50)
            temp_pdp = []
            
            for grid_val in temp_grid:
                X_modified = X_data.copy()
                X_modified[:, feature_idx] = grid_val
                temp_pred = model.predict(X_modified)
                temp_pdp.append(np.mean(temp_pred))
            
            temp_pdp = np.array(temp_pdp)
            
            # 计算PDP的梯度来找到异常跳跃
            if len(temp_grid) > 1:
                temp_gradient = np.gradient(temp_pdp, temp_grid)
                
                # 找到梯度绝对值最大的点
                max_gradient_idx = np.argmax(np.abs(temp_gradient))
                problem_value = temp_grid[max_gradient_idx]
                max_gradient = temp_gradient[max_gradient_idx]
                
                print(f"        最大梯度位置: {problem_value:.4f}, 梯度值: {max_gradient:.4f}")
                
                # 如果梯度过大，标记附近的样本为潜在异常值
                if np.abs(max_gradient) > np.std(temp_gradient) * 3:
                    tolerance = np.std(feature_data) * 0.1  # 使用特征标准差的10%作为容忍度
                    jump_outliers = np.abs(feature_data - problem_value) < tolerance
                    print(f"        标记 {np.sum(jump_outliers)} 个跳跃附近的样本")
                    return jump_outliers
            
            return np.zeros(len(feature_data), dtype=bool)
            
        except Exception as e:
            print(f"        PDP跳跃检测失败: {str(e)}")
            return np.zeros(len(feature_data), dtype=bool)
    
    def apply_cleaning(self, X_data, y_data):
        """应用数据清洗"""
        
        if self.global_outlier_mask is None:
            print("⚠️ 尚未执行异常值检测，返回原始数据")
            return X_data, y_data, np.array([])
        
        if not self.config.get('auto_clean', False):
            print("⚠️ 自动清洗已禁用，返回原始数据")
            return X_data, y_data, np.array([])
        
        outlier_count = np.sum(self.global_outlier_mask)
        if outlier_count == 0:
            print("✅ 未检测到异常值，返回原始数据")
            return X_data, y_data, np.array([])
        
        # 创建清洗后的数据
        clean_mask = ~self.global_outlier_mask
        X_cleaned = X_data[clean_mask]
        y_cleaned = y_data[clean_mask]
        removed_indices = np.where(self.global_outlier_mask)[0]
        
        print(f"🧹 数据清洗完成:")
        print(f"  原始样本数: {len(X_data)}")
        print(f"  移除异常值: {outlier_count}")
        print(f"  清洗后样本数: {len(X_cleaned)}")
        print(f"  数据保留率: {len(X_cleaned)/len(X_data)*100:.2f}%")
        
        return X_cleaned, y_cleaned, removed_indices
    
    def save_global_outlier_report(self, model_dir, features):
        """保存全局异常值检测报告"""
        
        if not self.global_outlier_report:
            print("⚠️ 没有全局异常值报告可保存")
            return False
        
        try:
            report = self.global_outlier_report
            
            # 保存全局异常值汇总
            global_summary = []
            for i, feature in enumerate(features):
                outlier_count = np.sum(report['outlier_mask'])
                global_summary.append({
                    '特征': feature,
                    '是否异常值': report['outlier_mask'],
                    '投票计数': report['vote_counts'],
                    '异常值总数': outlier_count,
                    '异常值比例': outlier_count / len(report['vote_counts']) * 100
                })
            
            # 保存详细的方法结果
            methods_results = {}
            for method_name, mask in report['all_methods'].items():
                methods_results[method_name] = mask
            
            # 保存到Excel
            excel_path = os.path.join(model_dir, 'outlier_detection', 'Global_Outlier_Report.xlsx')
            
            with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
                # 全局汇总
                global_df = pd.DataFrame({
                    'sample_index': np.arange(len(report['vote_counts'])),
                    'is_global_outlier': report['outlier_mask'],
                    'vote_counts': report['vote_counts']
                })
                global_df.to_excel(writer, sheet_name='Global_Summary', index=False)
                
                # 各方法详细结果
                methods_df = pd.DataFrame(methods_results)
                methods_df.insert(0, 'sample_index', np.arange(len(report['vote_counts'])))
                methods_df.to_excel(writer, sheet_name='All_Methods', index=False)
                
                # 统计信息
                stats_data = []
                for method_name, mask in report['all_methods'].items():
                    stats_data.append({
                        'method': method_name,
                        'outliers_detected': np.sum(mask),
                        'percentage': np.sum(mask) / len(mask) * 100
                    })
                
                stats_data.append({
                    'method': 'GLOBAL_CONSENSUS',
                    'outliers_detected': np.sum(report['outlier_mask']),
                    'percentage': report['outlier_percentage']
                })
                
                pd.DataFrame(stats_data).to_excel(writer, sheet_name='Statistics', index=False)
            
            print(f"✅ 全局异常值检测报告已保存: {excel_path}")
            return True
            
        except Exception as e:
            print(f"❌ 全局异常值检测报告保存失败: {str(e)}")
            return False

def load_gbdt_model_and_data():
    """加载GBDT模型和数据 - 修改：从训练程序的输出加载"""
    print("=== 加载GBDT模型和数据 ===")
    
    # 确保matplotlib设置正确
    plt.ioff()
    matplotlib.use('Agg')
    
    try:
        # 加载训练好的GBDT模型
        model_file = os.path.join(CONFIG['gbdt_model_dir'], f"optimal_GBDT_model_{CONFIG['model_timestamp']}.pkl")
        with open(model_file, 'rb') as f:
            gbdt_model = pickle.load(f)
        print(f"✓ 成功加载GBDT模型: {model_file}")
        
        # 加载模型信息
        info_file = os.path.join(CONFIG['gbdt_model_dir'], f"model_info_{CONFIG['model_timestamp']}.pkl")
        with open(info_file, 'rb') as f:
            model_info_data = pickle.load(f)
        print(f"✓ 成功加载模型信息: {info_file}")
        
        # 加载原始数据
        df = pd.read_excel(CONFIG['data_file'])
        print(f"✓ 成功加载原始数据: {df.shape}")
        
        # 提取特征和目标变量 - 确保与训练时一致
        X_data = df[CONFIG['selected_features']].copy()
        y_data = df[CONFIG['target_col']].copy()
        
        # 检查并移除包含NaN的行 - 与训练程序保持一致
        nan_mask_x = ~X_data.isnull().any(axis=1)
        nan_mask_y = ~y_data.isnull()
        valid_mask = nan_mask_x & nan_mask_y
        
        X_clean = X_data[valid_mask].values
        y_clean = y_data[valid_mask].values
        
        print(f"✓ 数据清理完成: {X_clean.shape}")
        print(f"✓ 特征: {CONFIG['selected_features']}")
        print(f"✓ 目标变量: {CONFIG['target_col']}")
        
        # 构造模型信息字典 - 适配原PDP程序的格式
        model_info = {
            'model_key': 'GBDT_PCC_k5_Optimal',
            'method': 'PCC',
            'k': 5,
            'model': 'GradientBoosting',
            'features': CONFIG['selected_features'],
            'model_instance': gbdt_model,
            'X_data': X_clean,
            'y_data': y_clean,
            'train_r2': model_info_data['metrics']['Train R²'],
            'test_r2': model_info_data['metrics']['Test R²'],
            'train_mae': model_info_data['metrics']['Train MAE'],
            'test_mae': model_info_data['metrics']['Test MAE'],
            'feature_importance': model_info_data['feature_importance'],
            'cv_scores': np.array(model_info_data['cv_scores']),
            'model_params': model_info_data['model_parameters']
        }
        
        # 返回单个模型的列表（适配原程序处理多个模型的结构）
        best_models = [model_info]
        
        # 构造其他必要的返回值
        all_features_name = CONFIG['selected_features']
        scaler = None  # GBDT不需要标准化
        data_info = {
            'original_shape': df.shape,
            'cleaned_shape': X_clean.shape,
            'features': CONFIG['selected_features'],
            'target': CONFIG['target_col']
        }
        
        print(f"✓ 成功构造模型信息，准备进行PDP分析")
        
        return best_models, all_features_name, scaler, data_info
        
    except Exception as e:
        print(f"❌ 加载GBDT模型和数据失败: {str(e)}")
        raise e

def create_model_directory(model_info):
    """为GBDT模型创建独立的输出目录"""
    model_name = f"{model_info['method']}_k{model_info['k']}_{model_info['model']}_Optimal"
    # 清理文件名中的非法字符
    model_name = model_name.replace(' ', '_').replace('-', '_').replace('/', '_')
    
    model_dir = os.path.join(CONFIG['pdp_output_dir'], model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # 创建子目录
    subdirs = [
        'combined_pdp',           # 合并的PDP分析（individual + confidence）
        'interaction_2d_pdp',     # 2D特征交互PDP
        'interaction_3d_pdp',     # 3D特征交互PDP
        'outlier_detection',      # 异常值检测报告
        'cleaned_analysis',       # 清洗后的分析结果
    ]
    for subdir in subdirs:
        os.makedirs(os.path.join(model_dir, subdir), exist_ok=True)
    
    return model_dir, model_name

def safe_filename(name):
    """生成安全的文件名，移除所有可能的非法字符"""
    # 替换常见的非法字符
    replacements = {
        ' ': '_',
        '/': '_',
        '\\': '_',
        ':': '_',
        '*': '_',
        '?': '_',
        '"': '_',
        '<': '_',
        '>': '_',
        '|': '_',
        '(': '',
        ')': '',
        '[': '',
        ']': '',
        '{': '',
        '}': '',
        '×': 'x',
        '÷': 'div',
        '+': 'plus',
        '=': 'eq',
        '%': 'percent',
        '#': 'num',
        '&': 'and',
        '@': 'at',
        '$': 'dollar',
        '!': 'excl',
        '^': 'caret',
        '~': 'tilde',
        '`': 'grave',
        "'": '',
        ',': '_',
        ';': '_',
        '.': '_',
        '-': '_'
    }
    
    safe_name = name
    for old, new in replacements.items():
        safe_name = safe_name.replace(old, new)
    
    # 移除连续的下划线
    while '__' in safe_name:
        safe_name = safe_name.replace('__', '_')
    
    # 移除开头和结尾的下划线
    safe_name = safe_name.strip('_')
    
    # 限制长度（Excel工作表名称限制31字符）
    if len(safe_name) > 25:
        safe_name = safe_name[:25]
    
    return safe_name

def save_data_to_excel(data, filepath, sheet_name='Data', additional_info=None):
    """改进的Excel保存函数，更好地处理各种数据类型"""
    try:
        with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
            # 保存主要数据
            if isinstance(data, dict):
                for key, value in data.items():
                    safe_sheet_name = safe_filename(key)[:31]  # Excel工作表名称限制31字符
                    
                    if isinstance(value, np.ndarray):
                        if value.ndim == 1:
                            # 1D数组直接保存
                            df = pd.DataFrame({safe_filename(key): value})
                            df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
                        elif value.ndim == 2:
                            # 2D数组保存为矩阵形式
                            df = pd.DataFrame(value)
                            df.to_excel(writer, sheet_name=safe_sheet_name, index=True)
                        else:
                            # 高维数组展平后保存
                            df = pd.DataFrame({safe_filename(key): value.flatten()})
                            df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
                    elif isinstance(value, (list, tuple)):
                        # 列表或元组
                        try:
                            df = pd.DataFrame({safe_filename(key): value})
                            df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
                        except:
                            # 如果直接转换失败，尝试其他方法
                            df = pd.DataFrame({'data': [str(v) for v in value]})
                            df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
                    elif isinstance(value, pd.DataFrame):
                        # DataFrame直接保存
                        value.to_excel(writer, sheet_name=safe_sheet_name, index=False)
                    else:
                        # 其他类型转换为字符串
                        df = pd.DataFrame({safe_filename(key): [str(value)]})
                        df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
                        
            elif isinstance(data, pd.DataFrame):
                safe_sheet_name = safe_filename(sheet_name)[:31]
                data.to_excel(writer, sheet_name=safe_sheet_name, index=False)
            elif isinstance(data, (list, np.ndarray)):
                safe_sheet_name = safe_filename(sheet_name)[:31]
                if isinstance(data, np.ndarray) and data.ndim > 1:
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame({safe_filename(sheet_name): data})
                df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
            
            # 保存额外信息
            if additional_info:
                try:
                    info_df = pd.DataFrame(list(additional_info.items()), columns=['项目', '值'])
                    info_df.to_excel(writer, sheet_name='信息', index=False)
                except:
                    # 如果额外信息保存失败，跳过但不影响主要数据
                    pass
        
        return True
    except Exception as e:
        print(f"      ❌ Excel保存失败: {str(e)}")
        return False

def save_2d_interaction_data(feature1_values, feature2_values, Z, feature1_name, feature2_name, excel_path, additional_info=None):
    """专门用于保存2D交互数据的函数"""
    try:
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # 保存特征1的值
            df1 = pd.DataFrame({'feature1_values': feature1_values})
            df1.to_excel(writer, sheet_name='Feature1_Values', index=False)
            
            # 保存特征2的值
            df2 = pd.DataFrame({'feature2_values': feature2_values})
            df2.to_excel(writer, sheet_name='Feature2_Values', index=False)
            
            # 保存2D PDP矩阵
            df_matrix = pd.DataFrame(Z)
            df_matrix.columns = [f'F1_{i:.4f}' for i in range(len(feature1_values))]
            df_matrix.index = [f'F2_{i:.4f}' for i in range(len(feature2_values))]
            df_matrix.to_excel(writer, sheet_name='PDP_Matrix', index=True)
            
            # 保存为长格式数据（便于后续分析）
            XX, YY = np.meshgrid(feature1_values, feature2_values)
            long_data = []
            for i in range(Z.shape[0]):
                for j in range(Z.shape[1]):
                    long_data.append({
                        'feature1_value': XX[i, j],
                        'feature2_value': YY[i, j],
                        'pdp_value': Z[i, j],
                        'feature1_index': j,
                        'feature2_index': i
                    })
            
            df_long = pd.DataFrame(long_data)
            df_long.to_excel(writer, sheet_name='Long_Format', index=False)
            
            # 保存额外信息
            if additional_info:
                info_df = pd.DataFrame(list(additional_info.items()), columns=['项目', '值'])
                info_df.to_excel(writer, sheet_name='信息', index=False)
        
        return True
    except Exception as e:
        print(f"      ❌ 2D交互数据Excel保存失败: {str(e)}")
        return False

def save_3d_interaction_data(feature1_values, feature2_values, feature3_values, Z_3d, feature1_name, feature2_name, feature3_name, excel_path, additional_info=None):
    """专门用于保存3D交互数据的函数"""
    try:
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # 保存特征值
            df1 = pd.DataFrame({'feature1_values': feature1_values})
            df1.to_excel(writer, sheet_name='Feature1_Values', index=False)
            
            df2 = pd.DataFrame({'feature2_values': feature2_values})
            df2.to_excel(writer, sheet_name='Feature2_Values', index=False)
            
            df3 = pd.DataFrame({'feature3_values': feature3_values})
            df3.to_excel(writer, sheet_name='Feature3_Values', index=False)
            
            # 保存3D数据为扁平化格式
            flat_3d_data = []
            for i, val1 in enumerate(feature1_values):
                for j, val2 in enumerate(feature2_values):
                    for k, val3 in enumerate(feature3_values):
                        flat_3d_data.append({
                            'feature1_value': val1,
                            'feature2_value': val2,
                            'feature3_value': val3,
                            'pdp_value': Z_3d[i, j, k],
                            'feature1_index': i,
                            'feature2_index': j,
                            'feature3_index': k
                        })
            
            df_3d_flat = pd.DataFrame(flat_3d_data)
            df_3d_flat.to_excel(writer, sheet_name='3D_Flat_Data', index=False)
            
            # 保存切片数据（固定一个特征维度）
            for slice_idx in [0, len(feature3_values)//2, -1]:
                if slice_idx == -1:
                    slice_idx = len(feature3_values) - 1
                
                slice_data = Z_3d[:, :, slice_idx]
                df_slice = pd.DataFrame(slice_data)
                df_slice.columns = [f'F2_{val:.4f}' for val in feature2_values]
                df_slice.index = [f'F1_{val:.4f}' for val in feature1_values]
                
                sheet_name = f'Slice_F3_{feature3_values[slice_idx]:.4f}'[:31]
                df_slice.to_excel(writer, sheet_name=sheet_name, index=True)
            
            # 保存额外信息
            if additional_info:
                info_df = pd.DataFrame(list(additional_info.items()), columns=['项目', '值'])
                info_df.to_excel(writer, sheet_name='信息', index=False)
        
        return True
    except Exception as e:
        print(f"      ❌ 3D交互数据Excel保存失败: {str(e)}")
        return False

def calculate_feature_importance(model, X_data, y_data, feature_names):
    """计算特征重要性"""
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        importance_type = "Built-in Feature Importance"
    else:
        # 使用置换重要性
        perm_importance = permutation_importance(model, X_data, y_data, n_repeats=5, random_state=42)
        importances = perm_importance.importances_mean
        importance_type = "Permutation Importance"
    
    return importances, importance_type

def compare_pdp_before_after_cleaning(model, X_original, X_cleaned, feature_idx, feature_name, 
                                     model_dir, outliers_removed):
    """对比清洗前后的PDP分析结果"""
    
    print(f"      生成清洗前后PDP对比分析...")
    
    try:
        # 原始数据的PDP分析
        feature_data_original = X_original[:, feature_idx]
        feature_min_orig = np.percentile(feature_data_original, 1)
        feature_max_orig = np.percentile(feature_data_original, 99)
        fixed_grid_orig = np.linspace(feature_min_orig, feature_max_orig, CONFIG['grid_resolution'])
        
        base_pdp_original = []
        for grid_val in fixed_grid_orig:
            X_modified = X_original.copy()
            X_modified[:, feature_idx] = grid_val
            predictions = model.predict(X_modified)
            base_pdp_original.append(np.mean(predictions))
        
        base_pdp_original = np.array(base_pdp_original)
        
        # 清洗后数据的PDP分析
        feature_data_cleaned = X_cleaned[:, feature_idx]
        feature_min_clean = np.percentile(feature_data_cleaned, 1)
        feature_max_clean = np.percentile(feature_data_cleaned, 99)
        fixed_grid_clean = np.linspace(feature_min_clean, feature_max_clean, CONFIG['grid_resolution'])
        
        base_pdp_cleaned = []
        for grid_val in fixed_grid_clean:
            X_modified = X_cleaned.copy()
            X_modified[:, feature_idx] = grid_val
            predictions = model.predict(X_modified)
            base_pdp_cleaned.append(np.mean(predictions))
        
        base_pdp_cleaned = np.array(base_pdp_cleaned)
        
        # 创建对比图
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. PDP曲线对比
        ax = axes[0, 0]
        ax.plot(fixed_grid_orig, base_pdp_original, 'red', linewidth=3, alpha=0.8, 
                label=f'清洗前PDP (n={len(X_original)})')
        ax.plot(fixed_grid_clean, base_pdp_cleaned, 'blue', linewidth=3, 
                label=f'清洗后PDP (n={len(X_cleaned)})')
        
        ax.set_xlabel(feature_name, fontsize=12, fontweight='bold')
        ax.set_ylabel('偏依赖值', fontsize=12, fontweight='bold')
        ax.set_title('清洗前后PDP曲线对比', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 2. 特征分布对比
        ax = axes[0, 1]
        ax.hist(feature_data_original, bins=50, alpha=0.6, color='red', 
                label=f'清洗前 (n={len(feature_data_original)})', density=True)
        ax.hist(feature_data_cleaned, bins=50, alpha=0.6, color='blue', 
                label=f'清洗后 (n={len(feature_data_cleaned)})', density=True)
        
        ax.set_xlabel(feature_name, fontsize=12, fontweight='bold')
        ax.set_ylabel('密度', fontsize=12, fontweight='bold')
        ax.set_title('特征分布对比', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 3. PDP斜率对比
        ax = axes[1, 0]
        if len(fixed_grid_orig) > 1:
            grad_orig = np.gradient(base_pdp_original, fixed_grid_orig)
            ax.plot(fixed_grid_orig, grad_orig, 'red', linewidth=2, alpha=0.8, label='清洗前斜率')
        
        if len(fixed_grid_clean) > 1:
            grad_clean = np.gradient(base_pdp_cleaned, fixed_grid_clean)
            ax.plot(fixed_grid_clean, grad_clean, 'blue', linewidth=2, label='清洗后斜率')
        
        ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        ax.set_xlabel(feature_name, fontsize=12, fontweight='bold')
        ax.set_ylabel('PDP斜率', fontsize=12, fontweight='bold')
        ax.set_title('PDP斜率对比', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 4. 统计对比表
        ax = axes[1, 1]
        
        # 计算统计指标
        pdp_range_orig = np.max(base_pdp_original) - np.min(base_pdp_original)
        pdp_range_clean = np.max(base_pdp_cleaned) - np.min(base_pdp_cleaned)
        
        feature_range_orig = np.max(feature_data_original) - np.min(feature_data_original)
        feature_range_clean = np.max(feature_data_cleaned) - np.min(feature_data_cleaned)
        
        max_grad_orig = np.max(np.abs(grad_orig)) if 'grad_orig' in locals() else 0
        max_grad_clean = np.max(np.abs(grad_clean)) if 'grad_clean' in locals() else 0
        
        comparison_stats = [
            ['统计指标', '清洗前', '清洗后', '变化'],
            ['样本数量', f"{len(feature_data_original)}", f"{len(feature_data_cleaned)}", f"-{outliers_removed}"],
            ['PDP变化幅度', f"{pdp_range_orig:.4f}", f"{pdp_range_clean:.4f}", f"{pdp_range_clean-pdp_range_orig:+.4f}"],
            ['特征范围', f"{feature_range_orig:.4f}", f"{feature_range_clean:.4f}", f"{feature_range_clean-feature_range_orig:+.4f}"],
            ['最大斜率', f"{max_grad_orig:.4f}", f"{max_grad_clean:.4f}", f"{max_grad_clean-max_grad_orig:+.4f}"],
            ['特征均值', f"{np.mean(feature_data_original):.4f}", f"{np.mean(feature_data_cleaned):.4f}", 
             f"{np.mean(feature_data_cleaned)-np.mean(feature_data_original):+.4f}"],
            ['特征标准差', f"{np.std(feature_data_original):.4f}", f"{np.std(feature_data_cleaned):.4f}",
             f"{np.std(feature_data_cleaned)-np.std(feature_data_original):+.4f}"]
        ]
        
        table = ax.table(cellText=comparison_stats[1:],
                        colLabels=comparison_stats[0],
                        cellLoc='center',
                        loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1.2, 1.8)
        ax.axis('off')
        ax.set_title('清洗前后统计对比', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        plt.suptitle(f'{feature_name} 异常值清洗前后PDP对比分析', fontsize=16, fontweight='bold', y=0.98)
        
        # 保存对比图
        safe_name = safe_filename(feature_name)
        img_path = os.path.join(model_dir, 'cleaned_analysis', f'PDP_Comparison_{safe_name}.png')
        plt.savefig(img_path, dpi=CONFIG['dpi'], bbox_inches='tight')
        plt.close()
        
        # 保存对比数据
        comparison_data = {
            'original_grid': fixed_grid_orig,
            'original_pdp': base_pdp_original,
            'cleaned_grid': fixed_grid_clean,
            'cleaned_pdp': base_pdp_cleaned,
            'original_feature_data': feature_data_original,
            'cleaned_feature_data': feature_data_cleaned
        }
        
        if 'grad_orig' in locals():
            comparison_data['original_gradient'] = grad_orig
        if 'grad_clean' in locals():
            comparison_data['cleaned_gradient'] = grad_clean
        
        excel_path = os.path.join(model_dir, 'cleaned_analysis', f'PDP_Comparison_Data_{safe_name}.xlsx')
        save_data_to_excel(comparison_data, excel_path, 'PDP对比数据')
        
        print(f"        ✓ PDP清洗前后对比分析已保存")
        return True
        
    except Exception as e:
        print(f"        ❌ PDP清洗前后对比分析失败: {str(e)}")
        plt.close('all')
        return False

def generate_combined_pdp_analysis(model_info, model_dir, data_export):
    """生成合并的PDP分析 - 使用统一清洗后的数据"""
    print(f"  生成统一清洗PDP分析（Individual + 置信区间）...")
    
    model = model_info['model_instance']
    X_data = model_info['X_data_cleaned']  # 使用清洗后的数据
    y_data = model_info['y_data_cleaned']  # 使用清洗后的数据
    features = model_info['features']
    
    # 获取自适应配置
    adaptive_config = get_adaptive_config(X_data.shape[0])
    n_bootstrap = adaptive_config['bootstrap_samples']
    
    print(f"    数据集大小: {X_data.shape[0]} (已清洗)")
    print(f"    网格分辨率: {CONFIG['grid_resolution']}点 (光滑曲线)")
    print(f"    Bootstrap次数: {n_bootstrap} (学术标准)")
    
    # 计算特征重要性
    importances, importance_type = calculate_feature_importance(model, X_data, y_data, features)
    
    # 为每个特征生成合并的PDP分析
    combined_data_all = {}
    
    for i, feature in enumerate(features):
        print(f"    处理特征: {feature}")
        
        try:
            # 获取清洗后的特征数据
            feature_data = X_data[:, i]
            
            # 计算特征的直方图分布数据
            histogram_data = calculate_histogram_data(feature_data)
            
            # 计算特征的实际范围
            feature_min = np.percentile(feature_data, CONFIG['percentiles'][0] * 100)
            feature_max = np.percentile(feature_data, CONFIG['percentiles'][1] * 100)
            
            # 创建固定的特征值网格
            fixed_grid = np.linspace(feature_min, feature_max, CONFIG['grid_resolution'])
            
            print(f"      开始 {n_bootstrap} 次Bootstrap采样...")
            
            # Bootstrap采样计算置信区间
            bootstrap_pdps = []
            
            for b in range(n_bootstrap):
                try:
                    # 创建bootstrap样本
                    boot_indices = np.random.choice(X_data.shape[0], X_data.shape[0], replace=True)
                    X_boot = X_data[boot_indices]
                    
                    # 手动计算PDP，使用固定网格
                    boot_pdp_values = []
                    for grid_val in fixed_grid:
                        # 创建修改后的样本
                        X_modified = X_boot.copy()
                        X_modified[:, i] = grid_val
                        
                        # 预测并取平均
                        predictions = model.predict(X_modified)
                        boot_pdp_values.append(np.mean(predictions))
                    
                    bootstrap_pdps.append(boot_pdp_values)
                    
                    # 进度提示（每200次）
                    if (b + 1) % 200 == 0:
                        print(f"        Bootstrap进度: {b + 1}/{n_bootstrap}")
                        
                except Exception as e:
                    print(f"      ⚠️ Bootstrap {b+1} 计算失败: {str(e)}")
                    continue
            
            if len(bootstrap_pdps) < max(10, n_bootstrap // 10):
                print(f"      ❌ 特征 {feature} Bootstrap样本不足，使用基础PDP")
                bootstrap_pdps = []
            
            # 重新计算基础PDP使用相同的固定网格
            base_pdp_fixed = []
            for grid_val in fixed_grid:
                X_modified = X_data.copy()
                X_modified[:, i] = grid_val
                predictions = model.predict(X_modified)
                base_pdp_fixed.append(np.mean(predictions))
            
            base_pdp_fixed = np.array(base_pdp_fixed)
            
            # 计算置信区间
            if len(bootstrap_pdps) > 0:
                bootstrap_pdps = np.array(bootstrap_pdps)
                success_rate = len(bootstrap_pdps) / n_bootstrap * 100
                
                # 计算多种置信区间
                confidence_intervals = {}
                for conf_level in [0.90, 0.95, 0.99]:
                    alpha = 1 - conf_level
                    lower_percentile = (alpha/2) * 100
                    upper_percentile = (1 - alpha/2) * 100
                    
                    ci_lower = np.percentile(bootstrap_pdps, lower_percentile, axis=0)
                    ci_upper = np.percentile(bootstrap_pdps, upper_percentile, axis=0)
                    
                    confidence_intervals[f'{int(conf_level*100)}%'] = {
                        'lower': ci_lower,
                        'upper': ci_upper,
                        'width': ci_upper - ci_lower
                    }
                
                # 使用主要置信水平
                main_ci = confidence_intervals[f'{int(CONFIG["confidence_level"]*100)}%']
                ci_lower = main_ci['lower']
                ci_upper = main_ci['upper']
                
                bootstrap_mean = np.mean(bootstrap_pdps, axis=0)
                bootstrap_std = np.std(bootstrap_pdps, axis=0)
                
                print(f"      成功Bootstrap: {len(bootstrap_pdps)}/{n_bootstrap} ({success_rate:.1f}%)")
            else:
                confidence_intervals = {}
                ci_lower = None
                ci_upper = None
                bootstrap_mean = None
                bootstrap_std = None
                success_rate = 0
                print(f"      未使用Bootstrap，仅显示基础PDP")
            
            # 保存合并数据
            combined_data_all[feature] = {
                'feature_values': fixed_grid,
                'base_pdp': base_pdp_fixed,
                'feature_importance': importances[i],
                'importance_type': importance_type,
                'bootstrap_count': len(bootstrap_pdps) if len(bootstrap_pdps) > 0 else 0,
                'success_rate': success_rate,
                'confidence_intervals': confidence_intervals,
                'ci_lower': ci_lower,
                'ci_upper': ci_upper,
                'bootstrap_mean': bootstrap_mean,
                'bootstrap_std': bootstrap_std,
                'cleaned_feature_data': feature_data,  # 清洗后特征数据
                'histogram_data': histogram_data,  # 直方图分布数据
                'data_suffix': "_cleaned"
            }
            
            # 创建增强的可视化图
            fig, axes = plt.subplots(3, 1, figsize=(CONFIG['figure_size'][0], CONFIG['figure_size'][1]*1.5))
            
            # 第一个子图：合并的PDP + 置信区间 + 特征分布
            ax1 = axes[0]
            
            # 绘制置信区间（如果有）
            if ci_lower is not None and ci_upper is not None:
                # 绘制多种置信区间（不同透明度）
                colors = ['lightblue', 'lightgreen', 'lightcoral']
                alphas = [0.6, 0.4, 0.2]
                for idx, (level, ci_data) in enumerate(confidence_intervals.items()):
                    ax1.fill_between(fixed_grid, ci_data['lower'], ci_data['upper'], 
                                   alpha=alphas[idx], color=colors[idx], 
                                   label=f'{level} 置信区间')
                
                # 绘制Bootstrap均值
                if bootstrap_mean is not None:
                    ax1.plot(fixed_grid, bootstrap_mean, 'red', linewidth=2, linestyle='--', 
                            alpha=0.8, label='Bootstrap均值')
            
            # 绘制主PDP曲线
            ax1.plot(fixed_grid, base_pdp_fixed, 'blue', linewidth=4, label='PDP曲线', zorder=10)
            
            # 添加特征分布（直方图，双Y轴）
            ax1_twin = ax1.twinx()
            ax1_twin.hist(feature_data, bins=CONFIG['histogram_bins'], alpha=0.3, 
                         color='gray', density=True, label='特征分布')
            ax1_twin.set_ylabel('特征密度', fontsize=12, color='gray')
            ax1_twin.tick_params(axis='y', labelcolor='gray')
            
            ax1.set_xlabel(feature, fontsize=14, fontweight='bold')
            ax1.set_ylabel('偏依赖值', fontsize=14, fontweight='bold')
            
            title_text = f'统一清洗PDP分析: {feature}\n重要性: {importances[i]:.4f} ({importance_type})'
            if len(bootstrap_pdps) > 0:
                title_text += f'\nBootstrap: {len(bootstrap_pdps)}次 (成功率: {success_rate:.1f}%)'
            title_text += f'\n使用统一清洗后数据'
            
            ax1.set_title(title_text, fontsize=14, fontweight='bold')
            ax1.legend(loc='upper left')
            ax1.grid(True, alpha=0.3)
            
            # 第二个子图：PDP一阶导数（斜率变化）
            ax2 = axes[1]
            if len(fixed_grid) > 1:
                pdp_gradient = np.gradient(base_pdp_fixed, fixed_grid)
                ax2.plot(fixed_grid, pdp_gradient, 'red', linewidth=3, label='PDP斜率')
                ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
                ax2.set_xlabel(feature, fontsize=12, fontweight='bold')
                ax2.set_ylabel('PDP斜率', fontsize=12, fontweight='bold')
                ax2.set_title('特征效应变化率', fontsize=12, fontweight='bold')
                ax2.grid(True, alpha=0.3)
                ax2.legend()
                
                max_gradient = np.max(np.abs(pdp_gradient))
            else:
                ax2.text(0.5, 0.5, '数据点不足\n无法计算斜率', ha='center', va='center', 
                        transform=ax2.transAxes, fontsize=14)
                max_gradient = 0
                pdp_gradient = np.array([])
            
            # 第三个子图：不确定性分析（如果有置信区间）
            ax3 = axes[2]
            if ci_lower is not None and ci_upper is not None:
                ci_width = ci_upper - ci_lower
                ax3.plot(fixed_grid, ci_width, 'purple', linewidth=3, label='置信区间宽度')
                if bootstrap_std is not None:
                    ax3.plot(fixed_grid, bootstrap_std, 'orange', linewidth=3, label='Bootstrap标准差')
                ax3.set_xlabel(feature, fontsize=12, fontweight='bold')
                ax3.set_ylabel('不确定性指标', fontsize=12, fontweight='bold')
                ax3.set_title('PDP不确定性分析', fontsize=12, fontweight='bold')
                ax3.legend()
                ax3.grid(True, alpha=0.3)
                
                avg_ci_width = np.mean(ci_width)
                max_uncertainty = np.max(ci_width)
            else:
                ax3.text(0.5, 0.5, '无Bootstrap数据\n无法分析不确定性', ha='center', va='center', 
                        transform=ax3.transAxes, fontsize=14)
                ax3.set_title('PDP不确定性分析', fontsize=12, fontweight='bold')
                avg_ci_width = 0
                max_uncertainty = 0
            
            # 添加统计信息文本框
            stats_text = f'网格分辨率: {CONFIG["grid_resolution"]}点\n' + \
                        f'分析样本数: {len(feature_data)} (清洗后)\n' + \
                        f'特征均值: {np.mean(feature_data):.4f}\n' + \
                        f'特征标准差: {np.std(feature_data):.4f}\n' + \
                        f'PDP变化幅度: {np.max(base_pdp_fixed) - np.min(base_pdp_fixed):.4f}\n' + \
                        f'最大斜率: {max_gradient:.4f}\n' + \
                        f'平均不确定性: {avg_ci_width:.4f}'
            
            ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes, 
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcyan", alpha=0.8),
                    verticalalignment='top', fontsize=9)
            
            plt.tight_layout()
            
            # 保存增强图片
            safe_name = safe_filename(feature)
            filename = f"Unified_Cleaned_PDP_Analysis_{safe_name}.png"
            img_path = os.path.join(model_dir, 'combined_pdp', filename)
            plt.savefig(img_path, dpi=CONFIG['dpi'], bbox_inches='tight')
            plt.close()
            
            # 保存对应的Excel数据
            excel_data = {
                'feature_values': fixed_grid,
                'base_pdp': base_pdp_fixed,
                'pdp_gradient': pdp_gradient if len(pdp_gradient) > 0 else np.full_like(fixed_grid, np.nan),
                
                # 清洗后特征分布数据
                'cleaned_feature_data': feature_data,
                'histogram_bin_centers': histogram_data['bin_centers'],
                'histogram_bin_edges': histogram_data['bin_edges'],
                'histogram_counts': histogram_data['counts'],
                'histogram_densities': histogram_data['densities'],
                'histogram_relative_frequencies': histogram_data['relative_frequencies'],
            }
            
            # 添加置信区间数据（如果有）
            if ci_lower is not None and ci_upper is not None:
                excel_data.update({
                    'bootstrap_mean': bootstrap_mean,
                    'bootstrap_std': bootstrap_std,
                    'ci_width': ci_width
                })
                
                # 添加所有置信区间
                for level, ci_data in confidence_intervals.items():
                    excel_data[f'ci_{level}_lower'] = ci_data['lower']
                    excel_data[f'ci_{level}_upper'] = ci_data['upper']
                    excel_data[f'ci_{level}_width'] = ci_data['width']
            
            excel_info = {
                '特征名称': feature,
                '特征重要性': importances[i],
                '重要性类型': importance_type,
                '网格分辨率': CONFIG['grid_resolution'],
                '分析样本数量': len(feature_data),
                '数据状态': '统一清洗后',
                '特征均值': np.mean(feature_data),
                '特征标准差': np.std(feature_data),
                '特征最小值': np.min(feature_data),
                '特征最大值': np.max(feature_data),
                '特征中位数': np.median(feature_data),
                'PDP变化幅度': np.max(base_pdp_fixed) - np.min(base_pdp_fixed),
                '最大斜率': max_gradient,
                'Bootstrap次数': len(bootstrap_pdps) if len(bootstrap_pdps) > 0 else 0,
                'Bootstrap成功率': success_rate,
                '平均不确定性': avg_ci_width,
                '最大不确定性': max_uncertainty,
                '直方图箱数': CONFIG['histogram_bins'],
                '直方图箱宽度': histogram_data['bin_width'],
                '数据覆盖范围': f"{feature_min:.4f} - {feature_max:.4f}",
                '统一清洗': '✅ 所有分析使用相同的清洗后数据'
            }
            
            excel_path = os.path.join(model_dir, 'combined_pdp', f"Unified_Cleaned_PDP_Analysis_{safe_name}.xlsx")
            save_data_to_excel(excel_data, excel_path, '统一清洗PDP数据', excel_info)
            
            print(f"      ✓ 特征 {feature} 统一清洗PDP分析和完整数据已保存")
            
        except Exception as e:
            print(f"      ❌ 特征 {feature} 统一清洗PDP分析失败: {str(e)}")
            import traceback
            traceback.print_exc()
            continue
        finally:
            plt.close('all')
    
    if not combined_data_all:
        print(f"    ❌ 所有特征统一清洗PDP计算都失败了")
        data_export['combined_pdp'] = {}
        return {}
    
    print(f"    ✓ 成功处理 {len(combined_data_all)}/{len(features)} 个特征")
    
    # 生成特征PDP综合对比图
    try:
        plt.figure(figsize=(18, 14))
        
        successful_features = list(combined_data_all.keys())
        n_features = len(successful_features)
        
        if n_features <= 4:
            rows, cols = 2, 2
        elif n_features <= 6:
            rows, cols = 2, 3
        elif n_features <= 9:
            rows, cols = 3, 3
        else:
            rows, cols = 4, 3
        
        comprehensive_data = {}
        
        for i, feature in enumerate(successful_features):
            if i >= rows * cols:
                break
                
            plt.subplot(rows, cols, i + 1)
            
            feature_values = combined_data_all[feature]['feature_values']
            base_pdp = combined_data_all[feature]['base_pdp']
            importance = combined_data_all[feature]['feature_importance']
            cleaned_data = combined_data_all[feature]['cleaned_feature_data']
            
            # 绘制置信区间（如果有）
            ci_lower = combined_data_all[feature]['ci_lower']
            ci_upper = combined_data_all[feature]['ci_upper']
            
            if ci_lower is not None and ci_upper is not None:
                plt.fill_between(feature_values, ci_lower, ci_upper, alpha=0.3, color='lightblue', label='95%置信区间')
            
            # 绘制主PDP曲线
            plt.plot(feature_values, base_pdp, 'blue', linewidth=2, label='PDP曲线')
            
            # 添加清洗后数据分布（右y轴）
            ax_twin = plt.gca().twinx()
            ax_twin.hist(cleaned_data, bins=20, alpha=0.2, color='gray', density=True)
            ax_twin.set_ylabel('密度', fontsize=8, color='gray')
            ax_twin.tick_params(axis='y', labelcolor='gray', labelsize=8)
            
            plt.xlabel(feature, fontsize=10)
            plt.ylabel('PDP', fontsize=10)
            
            title = f'{feature}\n重要性: {importance:.4f}'
            title += f'\n(统一清洗)'
            
            plt.title(title, fontsize=9, fontweight='bold')
            plt.grid(True, alpha=0.3)
            
            # 保存综合对比数据
            comprehensive_data[f'{safe_filename(feature)}_values'] = feature_values
            comprehensive_data[f'{safe_filename(feature)}_pdp'] = base_pdp
            comprehensive_data[f'{safe_filename(feature)}_cleaned_data'] = cleaned_data
        
        plt.suptitle(f'所有特征统一清洗PDP综合对比\n{model_info["method"]} + {model_info["model"]} (K={model_info["k"]})', 
                    fontsize=16, fontweight='bold')
        plt.tight_layout()
        
        # 保存综合对比图
        comp_img_path = os.path.join(model_dir, 'combined_pdp', 'All_Features_Unified_Cleaned_PDP_Comprehensive.png')
        plt.savefig(comp_img_path, dpi=CONFIG['dpi'], bbox_inches='tight')
        plt.close('all')
        
        # 保存综合对比数据
        comp_excel_path = os.path.join(model_dir, 'combined_pdp', 'All_Features_Unified_Cleaned_PDP_Comprehensive.xlsx')
        save_data_to_excel(comprehensive_data, comp_excel_path, '综合统一清洗PDP数据')
        
        print(f"    ✓ 统一清洗PDP综合对比图和数据已保存")
        
    except Exception as e:
        print(f"    ❌ 统一清洗PDP综合对比图生成失败: {str(e)}")
        plt.close('all')
    
    data_export['combined_pdp'] = combined_data_all
    return combined_data_all

def generate_complete_2d_interaction_pdp(model_info, model_dir, data_export):
    """生成完整的2D特征交互PDP分析 - 使用统一清洗后的数据"""
    print(f"  生成完整的2D特征交互PDP分析（使用统一清洗数据，所有特征对）...")
    
    model = model_info['model_instance']
    X_data = model_info['X_data_cleaned']  # 使用清洗后的数据
    features = model_info['features']
    
    interaction_data = {}
    
    # 生成所有可能的特征对组合
    feature_combinations = list(combinations(range(len(features)), 2))
    total_combinations = len(feature_combinations)
    
    print(f"    总共需要分析 {total_combinations} 个特征交互对")
    print(f"    2D网格分辨率: {CONFIG['interaction_2d_resolution']}×{CONFIG['interaction_2d_resolution']}")
    print(f"    使用统一清洗后数据: {X_data.shape[0]} 样本")
    
    successful_interactions = 0
    
    for comb_idx, (i, j) in enumerate(feature_combinations):
        feature1_name = features[i]
        feature2_name = features[j]
        
        print(f"    [{comb_idx+1}/{total_combinations}] 处理交互: {feature1_name} × {feature2_name}")
        
        try:
            # 计算2D PDP
            pd_result = partial_dependence(
                model, X_data, features=[i, j], 
                grid_resolution=CONFIG['interaction_2d_resolution'],
                percentiles=CONFIG['percentiles']
            )
            
            # 兼容新版sklearn的返回格式
            if hasattr(pd_result, 'grid_values'):
                feature1_values = pd_result.grid_values[0]
                feature2_values = pd_result.grid_values[1]
                Z_raw = pd_result.average
                
                if len(Z_raw.shape) == 3:
                    Z = Z_raw[0]
                elif len(Z_raw.shape) == 2:
                    Z = Z_raw
                else:
                    raise ValueError(f"Unexpected Z shape: {Z_raw.shape}")
            else:
                feature1_values = pd_result[1][0]
                feature2_values = pd_result[1][1]
                Z_raw = pd_result[0]
                
                if len(Z_raw.shape) == 3:
                    Z = Z_raw[0]
                else:
                    Z = Z_raw
            
            # 确保Z的形状正确
            expected_shape = (len(feature2_values), len(feature1_values))
            if Z.shape != expected_shape:
                if Z.shape == (len(feature1_values), len(feature2_values)):
                    Z = Z.T
                else:
                    raise ValueError(f"无法匹配Z矩阵形状: {Z.shape}")
            
            XX, YY = np.meshgrid(feature1_values, feature2_values)
            
            # 保存交互数据
            interaction_key = f"{feature1_name}_×_{feature2_name}"
            interaction_data[interaction_key] = {
                'feature1_values': feature1_values,
                'feature2_values': feature2_values, 
                'feature1_name': feature1_name,
                'feature2_name': feature2_name,
                'partial_dependence_2d': Z,
                'interaction_strength': np.var(Z)  # 使用方差衡量交互强度
            }
            
            # 创建增强的2D热力图
            fig, axes = plt.subplots(1, 2, figsize=(CONFIG['figure_size'][0]*1.5, CONFIG['figure_size'][1]))
            
            # 左图：热力图
            ax1 = axes[0]
            im1 = ax1.contourf(XX, YY, Z, levels=50, cmap='RdYlBu_r')
            fig.colorbar(im1, ax=ax1, label='偏依赖值')
            
            # 添加等高线
            contours = ax1.contour(XX, YY, Z, levels=10, colors='black', alpha=0.6, linewidths=0.8)
            ax1.clabel(contours, inline=True, fontsize=8, fmt='%.3f')
            
            ax1.set_xlabel(feature1_name, fontsize=12, fontweight='bold')
            ax1.set_ylabel(feature2_name, fontsize=12, fontweight='bold')
            ax1.set_title(f'2D交互热力图（统一清洗数据）\n{feature1_name} × {feature2_name}', fontsize=12, fontweight='bold')
            
            # 添加清洗后数据点分布
            feature1_data = X_data[:, i]
            feature2_data = X_data[:, j]
            ax1.scatter(feature1_data, feature2_data, c='white', s=0.5, alpha=0.3, label='清洗后数据点')
            ax1.legend()
            
            # 右图：3D表面图
            ax2 = fig.add_subplot(122, projection='3d')
            surf = ax2.plot_surface(XX, YY, Z, cmap='RdYlBu_r', alpha=0.9, 
                                 linewidth=0, antialiased=True)
            fig.colorbar(surf, ax=ax2, label='偏依赖值', shrink=0.5, aspect=20)
            
            ax2.set_xlabel(feature1_name, fontsize=10, fontweight='bold')
            ax2.set_ylabel(feature2_name, fontsize=10, fontweight='bold')
            ax2.set_zlabel('偏依赖值', fontsize=10, fontweight='bold')
            ax2.set_title(f'3D交互表面图（统一清洗数据）\n{feature1_name} × {feature2_name}', fontsize=10, fontweight='bold')
            
            plt.tight_layout()
            
            # 保存2D交互图
            safe_name1 = safe_filename(feature1_name)
            safe_name2 = safe_filename(feature2_name)
            filename = f"2D_Interaction_PDP_Cleaned_{safe_name1}_{safe_name2}.png"
            img_path = os.path.join(model_dir, 'interaction_2d_pdp', filename)
            plt.savefig(img_path, dpi=CONFIG['dpi'], bbox_inches='tight')
            plt.close('all')
            
            # 保存2D交互数据到Excel
            interaction_excel_info = {
                '特征1': feature1_name,
                '特征2': feature2_name,
                '网格分辨率': f"{Z.shape[0]}×{Z.shape[1]}",
                '最小PDP值': np.min(Z),
                '最大PDP值': np.max(Z),
                'PDP变化幅度': np.max(Z) - np.min(Z),
                '交互强度(方差)': np.var(Z),
                '交互排名': comb_idx + 1,
                '数据状态': '统一清洗后',
                '样本数量': X_data.shape[0]
            }
            
            excel_path = os.path.join(model_dir, 'interaction_2d_pdp', f"2D_Interaction_PDP_Cleaned_{safe_name1}_{safe_name2}.xlsx")
            success = save_2d_interaction_data(feature1_values, feature2_values, Z, feature1_name, feature2_name, excel_path, interaction_excel_info)
            
            if success:
                print(f"      ✓ 2D交互 {feature1_name} × {feature2_name} 分析和数据保存成功")
            else:
                print(f"      ⚠️ 2D交互 {feature1_name} × {feature2_name} 图片保存成功，数据保存失败")
            
            successful_interactions += 1
                
        except Exception as e:
            print(f"      ❌ 交互 {feature1_name} × {feature2_name} 分析失败: {str(e)}")
            plt.close('all')
            continue
    
    # 生成交互强度排序报告
    if interaction_data:
        try:
            print(f"    生成交互强度排序报告...")
            
            # 按交互强度排序
            sorted_interactions = sorted(interaction_data.items(), 
                                       key=lambda x: x[1]['interaction_strength'], 
                                       reverse=True)
            
            # 创建交互强度对比图
            plt.figure(figsize=(16, 10))
            
            # 取前10个最强交互
            top_interactions = sorted_interactions[:min(10, len(sorted_interactions))]
            
            interaction_names = [name.replace('_×_', ' × ') for name, _ in top_interactions]
            interaction_strengths = [data['interaction_strength'] for _, data in top_interactions]
            
            bars = plt.bar(range(len(interaction_names)), interaction_strengths, 
                          color=plt.cm.viridis(np.linspace(0, 1, len(interaction_names))))
            
            plt.xlabel('特征交互对', fontsize=14, fontweight='bold')
            plt.ylabel('交互强度 (PDP方差)', fontsize=14, fontweight='bold')
            plt.title(f'Top {len(top_interactions)} 最强2D特征交互（统一清洗数据）\n总共分析了 {len(interaction_data)} 个交互对', 
                     fontsize=16, fontweight='bold')
            plt.xticks(range(len(interaction_names)), interaction_names, rotation=45, ha='right')
            plt.grid(True, alpha=0.3, axis='y')
            
            # 添加数值标签
            for i, (bar, strength) in enumerate(zip(bars, interaction_strengths)):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(interaction_strengths)*0.01, 
                        f'{strength:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
            
            plt.tight_layout()
            
            # 保存交互强度排序图
            strength_img_path = os.path.join(model_dir, 'interaction_2d_pdp', 'Interaction_Strength_Ranking_Cleaned.png')
            plt.savefig(strength_img_path, dpi=CONFIG['dpi'], bbox_inches='tight')
            plt.close('all')
            
            # 保存交互强度排序数据
            ranking_data = []
            for rank, (name, data) in enumerate(sorted_interactions, 1):
                ranking_data.append({
                    '排名': rank,
                    '特征交互对': name.replace('_×_', ' × '),
                    '特征1': data['feature1_name'],
                    '特征2': data['feature2_name'],
                    '交互强度': data['interaction_strength'],
                    '最小PDP值': np.min(data['partial_dependence_2d']),
                    '最大PDP值': np.max(data['partial_dependence_2d']),
                    'PDP变化幅度': np.max(data['partial_dependence_2d']) - np.min(data['partial_dependence_2d']),
                    '数据状态': '统一清洗后'
                })
            
            ranking_df = pd.DataFrame(ranking_data)
            ranking_excel_path = os.path.join(model_dir, 'interaction_2d_pdp', 'All_2D_Interactions_Ranking_Cleaned.xlsx')
            ranking_df.to_excel(ranking_excel_path, index=False)
            
            print(f"      ✓ 交互强度排序报告已保存")
            
        except Exception as e:
            print(f"      ❌ 交互强度排序报告生成失败: {str(e)}")
            plt.close('all')
    
    print(f"    ✓ 完整2D交互分析完成：成功处理 {successful_interactions}/{total_combinations} 个交互对")
    data_export['interaction_2d_pdp'] = interaction_data
    return interaction_data

def generate_selected_3d_interaction_pdp(model_info, model_dir, data_export, max_3d_interactions=None):
    """生成选定的3D特征交互PDP分析 - 使用统一清洗后的数据"""
    print(f"  生成选定的3D特征交互PDP分析（使用统一清洗数据）...")
    
    model = model_info['model_instance']
    X_data = model_info['X_data_cleaned']  # 使用清洗后的数据
    y_data = model_info['y_data_cleaned']  # 使用清洗后的数据
    features = model_info['features']
    
    if max_3d_interactions is None:
        max_3d_interactions = CONFIG['max_3d_interactions']
    
    # 计算特征重要性来选择最重要的特征进行3D分析
    importances, _ = calculate_feature_importance(model, X_data, y_data, features)
    
    # 选择重要性最高的特征进行3D交互分析
    top_feature_indices = np.argsort(importances)[-min(6, len(features)):]  # 最多取6个最重要的特征
    top_features = [features[i] for i in top_feature_indices]
    
    # 生成3D组合（限制数量）
    feature_3d_combinations = list(combinations(top_feature_indices, 3))
    
    # 限制3D交互数量
    if len(feature_3d_combinations) > max_3d_interactions:
        # 优先选择包含最重要特征的组合
        feature_3d_combinations = feature_3d_combinations[:max_3d_interactions]
    
    total_3d_combinations = len(feature_3d_combinations)
    
    print(f"    选择了前 {len(top_features)} 个最重要特征进行3D分析")
    print(f"    总共需要分析 {total_3d_combinations} 个3D特征交互")
    print(f"    3D网格分辨率: {CONFIG['interaction_3d_resolution']}³")
    print(f"    使用统一清洗后数据: {X_data.shape[0]} 样本")
    
    if total_3d_combinations == 0:
        print(f"    ⚠️ 没有足够的特征进行3D交互分析")
        data_export['interaction_3d_pdp'] = {}
        return {}
    
    interaction_3d_data = {}
    successful_3d_interactions = 0
    
    for comb_idx, (i, j, k) in enumerate(feature_3d_combinations):
        feature1_name = features[i]
        feature2_name = features[j]
        feature3_name = features[k]
        
        print(f"    [{comb_idx+1}/{total_3d_combinations}] 处理3D交互: {feature1_name} × {feature2_name} × {feature3_name}")
        
        try:
            # 手动计算3D PDP（sklearn的partial_dependence只支持1D和2D）
            feature1_data = X_data[:, i]
            feature2_data = X_data[:, j]
            feature3_data = X_data[:, k]
            
            # 创建3D网格
            feature1_range = np.linspace(np.percentile(feature1_data, 1), np.percentile(feature1_data, 99), CONFIG['interaction_3d_resolution'])
            feature2_range = np.linspace(np.percentile(feature2_data, 1), np.percentile(feature2_data, 99), CONFIG['interaction_3d_resolution'])
            feature3_range = np.linspace(np.percentile(feature3_data, 1), np.percentile(feature3_data, 99), CONFIG['interaction_3d_resolution'])
            
            # 计算3D PDP
            Z_3d = np.zeros((len(feature1_range), len(feature2_range), len(feature3_range)))
            
            total_grid_points = len(feature1_range) * len(feature2_range) * len(feature3_range)
            print(f"      计算 {total_grid_points} 个3D网格点...")
            
            for idx1, val1 in enumerate(feature1_range):
                for idx2, val2 in enumerate(feature2_range):
                    for idx3, val3 in enumerate(feature3_range):
                        X_modified = X_data.copy()
                        X_modified[:, i] = val1
                        X_modified[:, j] = val2
                        X_modified[:, k] = val3
                        
                        predictions = model.predict(X_modified)
                        Z_3d[idx1, idx2, idx3] = np.mean(predictions)
                
                # 进度提示
                if (idx1 + 1) % 5 == 0:
                    progress = ((idx1 + 1) * len(feature2_range) * len(feature3_range)) / total_grid_points * 100
                    print(f"        3D计算进度: {progress:.1f}%")
            
            # 保存3D交互数据
            interaction_key = f"{feature1_name}_×_{feature2_name}_×_{feature3_name}"
            interaction_3d_data[interaction_key] = {
                'feature1_values': feature1_range,
                'feature2_values': feature2_range,
                'feature3_values': feature3_range,
                'feature1_name': feature1_name,
                'feature2_name': feature2_name,
                'feature3_name': feature3_name,
                'partial_dependence_3d': Z_3d,
                'interaction_strength_3d': np.var(Z_3d)
            }
            
            # 创建3D可视化（多个切片图）
            fig, axes = plt.subplots(2, 3, figsize=(18, 12))
            axes = axes.ravel()
            
            # 生成6个不同的切片
            slice_indices = [0, len(feature3_range)//4, len(feature3_range)//2, 
                           3*len(feature3_range)//4, len(feature3_range)-1]
            slice_indices = slice_indices[:6]  # 确保只有6个切片
            
            for slice_idx, ax in enumerate(axes):
                if slice_idx < len(slice_indices):
                    f3_idx = slice_indices[slice_idx]
                    f3_val = feature3_range[f3_idx]
                    
                    # 创建2D网格用于这个切片
                    XX, YY = np.meshgrid(feature2_range, feature1_range)
                    Z_slice = Z_3d[:, :, f3_idx]
                    
                    # 绘制热力图
                    im = ax.contourf(XX, YY, Z_slice, levels=20, cmap='RdYlBu_r')
                    fig.colorbar(im, ax=ax)
                    
                    # 添加等高线
                    contours = ax.contour(XX, YY, Z_slice, levels=8, colors='black', alpha=0.6, linewidths=0.5)
                    
                    ax.set_xlabel(feature2_name, fontsize=10)
                    ax.set_ylabel(feature1_name, fontsize=10)
                    ax.set_title(f'{feature3_name}={f3_val:.3f}', fontsize=10, fontweight='bold')
                else:
                    ax.axis('off')
            
            plt.suptitle(f'3D交互PDP切片图（统一清洗数据）: {feature1_name} × {feature2_name} × {feature3_name}', 
                        fontsize=16, fontweight='bold')
            plt.tight_layout()
            
            # 保存3D交互图
            safe_name1 = safe_filename(feature1_name)
            safe_name2 = safe_filename(feature2_name)
            safe_name3 = safe_filename(feature3_name)
            filename = f"3D_Interaction_PDP_Cleaned_{safe_name1}_{safe_name2}_{safe_name3}.png"
            img_path = os.path.join(model_dir, 'interaction_3d_pdp', filename)
            plt.savefig(img_path, dpi=CONFIG['dpi'], bbox_inches='tight')
            plt.close('all')
            
            # 保存3D交互数据到Excel
            interaction_3d_excel_info = {
                '特征1': feature1_name,
                '特征2': feature2_name,
                '特征3': feature3_name,
                '3D网格分辨率': f"{Z_3d.shape[0]}×{Z_3d.shape[1]}×{Z_3d.shape[2]}",
                '最小PDP值': np.min(Z_3d),
                '最大PDP值': np.max(Z_3d),
                'PDP变化幅度': np.max(Z_3d) - np.min(Z_3d),
                '3D交互强度': np.var(Z_3d),
                '3D交互排名': comb_idx + 1,
                '数据点总数': Z_3d.size,
                '数据状态': '统一清洗后',
                '样本数量': X_data.shape[0]
            }
            
            excel_path = os.path.join(model_dir, 'interaction_3d_pdp', f"3D_Interaction_PDP_Cleaned_{safe_name1}_{safe_name2}_{safe_name3}.xlsx")
            success = save_3d_interaction_data(feature1_range, feature2_range, feature3_range, Z_3d, 
                                             feature1_name, feature2_name, feature3_name, excel_path, interaction_3d_excel_info)
            
            if success:
                print(f"      ✓ 3D交互 {feature1_name} × {feature2_name} × {feature3_name} 分析和数据保存成功")
            else:
                print(f"      ⚠️ 3D交互 {feature1_name} × {feature2_name} × {feature3_name} 图片保存成功，数据保存失败")
            
            successful_3d_interactions += 1
                
        except Exception as e:
            print(f"      ❌ 3D交互 {feature1_name} × {feature2_name} × {feature3_name} 分析失败: {str(e)}")
            plt.close('all')
            continue
    
    # 生成3D交互强度排序报告
    if interaction_3d_data:
        try:
            print(f"    生成3D交互强度排序报告...")
            
            # 按3D交互强度排序
            sorted_3d_interactions = sorted(interaction_3d_data.items(), 
                                          key=lambda x: x[1]['interaction_strength_3d'], 
                                          reverse=True)
            
            # 创建3D交互强度对比图
            plt.figure(figsize=(14, 8))
            
            interaction_3d_names = [name.replace('_×_', ' × ') for name, _ in sorted_3d_interactions]
            interaction_3d_strengths = [data['interaction_strength_3d'] for _, data in sorted_3d_interactions]
            
            bars = plt.bar(range(len(interaction_3d_names)), interaction_3d_strengths, 
                          color=plt.cm.plasma(np.linspace(0, 1, len(interaction_3d_names))))
            
            plt.xlabel('3D特征交互组合', fontsize=14, fontweight='bold')
            plt.ylabel('3D交互强度 (PDP方差)', fontsize=14, fontweight='bold')
            plt.title(f'所有3D特征交互强度排序（统一清洗数据）\n成功分析了 {len(interaction_3d_data)} 个3D交互', 
                     fontsize=16, fontweight='bold')
            plt.xticks(range(len(interaction_3d_names)), interaction_3d_names, rotation=45, ha='right')
            plt.grid(True, alpha=0.3, axis='y')
            
            # 添加数值标签
            for i, (bar, strength) in enumerate(zip(bars, interaction_3d_strengths)):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(interaction_3d_strengths)*0.01, 
                        f'{strength:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
            
            plt.tight_layout()
            
            # 保存3D交互强度排序图
            strength_3d_img_path = os.path.join(model_dir, 'interaction_3d_pdp', '3D_Interaction_Strength_Ranking_Cleaned.png')
            plt.savefig(strength_3d_img_path, dpi=CONFIG['dpi'], bbox_inches='tight')
            plt.close('all')
            
            # 保存3D交互强度排序数据
            ranking_3d_data = []
            for rank, (name, data) in enumerate(sorted_3d_interactions, 1):
                ranking_3d_data.append({
                    '排名': rank,
                    '3D特征交互组合': name.replace('_×_', ' × '),
                    '特征1': data['feature1_name'],
                    '特征2': data['feature2_name'],
                    '特征3': data['feature3_name'],
                    '3D交互强度': data['interaction_strength_3d'],
                    '最小PDP值': np.min(data['partial_dependence_3d']),
                    '最大PDP值': np.max(data['partial_dependence_3d']),
                    'PDP变化幅度': np.max(data['partial_dependence_3d']) - np.min(data['partial_dependence_3d']),
                    '数据点总数': data['partial_dependence_3d'].size,
                    '数据状态': '统一清洗后'
                })
            
            ranking_3d_df = pd.DataFrame(ranking_3d_data)
            ranking_3d_excel_path = os.path.join(model_dir, 'interaction_3d_pdp', 'All_3D_Interactions_Ranking_Cleaned.xlsx')
            ranking_3d_df.to_excel(ranking_3d_excel_path, index=False)
            
            print(f"      ✓ 3D交互强度排序报告已保存")
            
        except Exception as e:
            print(f"      ❌ 3D交互强度排序报告生成失败: {str(e)}")
            plt.close('all')
    
    print(f"    ✓ 3D交互分析完成：成功处理 {successful_3d_interactions}/{total_3d_combinations} 个3D交互")
    data_export['interaction_3d_pdp'] = interaction_3d_data
    return interaction_3d_data

def analyze_gbdt_model():
    """分析GBDT模型 - 统一异常值处理版本"""
    print(f"\n[1/1] 分析GBDT模型: PCC_k=5_GradientBoosting")
    print(f"  特征: {', '.join(CONFIG['selected_features'])}")
    
    # 加载模型和数据
    best_models, all_features_name, scaler, data_info = load_gbdt_model_and_data()
    model_info = best_models[0]  # 只有一个模型
    
    print(f"  性能: R²={model_info['test_r2']:.6f}, MAE={model_info['test_mae']:.6f}")
    
    # 显示自适应配置
    adaptive_config = get_adaptive_config(model_info['X_data'].shape[0])
    print(f"  数据集大小: {model_info['X_data'].shape[0]}")
    print(f"  网格分辨率: {CONFIG['grid_resolution']}点 (高精度光滑曲线)")
    print(f"  自适应配置: Bootstrap {adaptive_config['bootstrap_samples']}次")
    print(f"  统一异常值处理: {'启用' if CONFIG['outlier_detection']['enable'] else '禁用'}")
    
    # 创建模型目录
    model_dir, model_name = create_model_directory(model_info)
    print(f"  输出目录: {model_dir}")
    
    # 存储所有导出数据
    data_export = {}
    
    try:
        print(f"\n开始统一异常值处理和PDP分析...")
        
        # === 步骤1: 统一异常值检测和数据清洗 ===
        X_original = model_info['X_data']
        y_original = model_info['y_data']
        features = model_info['features']
        model = model_info['model_instance']
        
        if CONFIG['outlier_detection']['enable']:
            print(f"\n1️⃣ 统一异常值检测和数据清洗")
            
            # 创建统一异常值检测器
            outlier_detector = UnifiedOutlierDetector(CONFIG['outlier_detection'])
            
            # 执行全局异常值检测
            global_outlier_mask, global_outlier_report = outlier_detector.detect_global_outliers(
                X_original, y_original, features, model
            )
            
            # 应用数据清洗
            X_cleaned, y_cleaned, removed_indices = outlier_detector.apply_cleaning(X_original, y_original)
            
            # 保存全局异常值报告
            outlier_detector.save_global_outlier_report(model_dir, features)
            
            # 更新模型信息，添加清洗后的数据
            model_info['X_data_cleaned'] = X_cleaned
            model_info['y_data_cleaned'] = y_cleaned
            model_info['removed_indices'] = removed_indices
            model_info['outlier_detector'] = outlier_detector
            
            outliers_removed = len(removed_indices)
            print(f"    统一异常值处理完成：移除 {outliers_removed} 个异常样本")
            
            # 生成清洗前后数据对比
            if outliers_removed > 0 and CONFIG['outlier_detection']['comparison_analysis']:
                print(f"    生成清洗前后数据对比分析...")
                for i, feature in enumerate(features):
                    compare_pdp_before_after_cleaning(
                        model, X_original, X_cleaned, i, feature, model_dir, outliers_removed
                    )
        else:
            print(f"\n1️⃣ 跳过异常值检测（已禁用）")
            # 如果禁用异常值检测，直接使用原始数据
            model_info['X_data_cleaned'] = X_original
            model_info['y_data_cleaned'] = y_original
            model_info['removed_indices'] = np.array([])
            model_info['outlier_detector'] = None
            outliers_removed = 0
        
        # === 步骤2: 使用统一清洗后数据进行所有PDP分析 ===
        print(f"\n2️⃣ 统一清洗数据PDP分析（Individual + 置信区间）")
        combined_data = generate_combined_pdp_analysis(model_info, model_dir, data_export)
        
        if not combined_data:
            print(f"统一清洗PDP分析失败，跳过后续分析")
            return False
        
        # === 步骤3: 使用统一清洗后数据进行2D交互分析 ===
        print(f"\n3️⃣ 统一清洗数据2D特征交互PDP分析（所有特征对）")
        interaction_2d_data = generate_complete_2d_interaction_pdp(model_info, model_dir, data_export)
        
        # === 步骤4: 使用统一清洗后数据进行3D交互分析 ===
        print(f"\n4️⃣ 统一清洗数据3D特征交互PDP分析（重要特征组合）")
        interaction_3d_data = generate_selected_3d_interaction_pdp(model_info, model_dir, data_export)
        
        print(f"\nGBDT模型 {model_name} 统一异常值处理分析完成！")
        
        # 输出分析结果统计
        print(f"\n分析结果统计:")
        print(f"• 原始样本数: {X_original.shape[0]}")
        print(f"• 异常值检测: {'已执行' if CONFIG['outlier_detection']['enable'] else '未执行'}")
        print(f"• 异常值清洗: 移除 {outliers_removed} 个样本")
        print(f"• 清洗后样本数: {model_info['X_data_cleaned'].shape[0]}")
        print(f"• 数据保留率: {model_info['X_data_cleaned'].shape[0]/X_original.shape[0]*100:.2f}%")
        print(f"• 分析特征数: {len(combined_data)}")
        print(f"• 网格分辨率: {CONFIG['grid_resolution']}点 (高精度)")
        print(f"• 2D交互分析: {len(interaction_2d_data)} 对 (使用统一清洗数据)")
        print(f"• 3D交互分析: {len(interaction_3d_data)} 组 (使用统一清洗数据)")
        print(f"• 数据一致性: 所有分析使用相同的清洗后数据")
        print(f"• 清洗前后对比: {'已生成' if CONFIG['outlier_detection']['comparison_analysis'] else '未生成'}")
        print(f"• 全局异常值报告: {'已生成' if CONFIG['outlier_detection']['save_outlier_reports'] else '未生成'}")
        
        return True
        
    except Exception as e:
        print(f"\nGBDT模型 {model_name} 统一异常值处理分析失败: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

def main():
    """主函数"""
    print("\n" + "="*80)
    print("GBDT模型PDP分析程序（统一异常值处理版）")
    print("="*80)
    
    # 初始清理
    cleanup_matplotlib()
    
    print(f"输出目录: {CONFIG['pdp_output_dir']}")
    print(f"分析类型: GBDT模型统一异常值处理 + 完整PDP和交互分析")
    print(f"matplotlib后端: {matplotlib.get_backend()}")
    print(f"交互模式: {'开启' if matplotlib.is_interactive() else '关闭'}")
    
    # 显示统一异常值处理配置
    print(f"\n统一异常值处理配置:")
    print(f"• 网格分辨率: {CONFIG['grid_resolution']}点 (Individual PDP)")
    print(f"• 2D交互分辨率: {CONFIG['interaction_2d_resolution']}×{CONFIG['interaction_2d_resolution']} (所有特征对)")
    print(f"• 3D交互分辨率: {CONFIG['interaction_3d_resolution']}³ (重要特征组合)")
    print(f"• 最大3D交互数: {CONFIG['max_3d_interactions']} (计算效率考虑)")
    print(f"• Bootstrap次数: 小数据集{CONFIG['bootstrap_samples_small']}, 中等{CONFIG['bootstrap_samples']}, 大型{CONFIG['bootstrap_samples_large']}")
    print(f"• 置信水平: {CONFIG['confidence_level']*100}%")
    print(f"• 直方图箱数: {CONFIG['histogram_bins']} (特征分布)")
    
    # 统一异常值检测配置
    outlier_config = CONFIG['outlier_detection']
    print(f"\n统一异常值检测配置:")
    print(f"• 异常值检测: {'启用' if outlier_config['enable'] else '禁用'}")
    if outlier_config['enable']:
        print(f"• 统一清洗: {'启用' if outlier_config['unified_cleaning'] else '禁用'}")
        print(f"• 自动清洗: {'是' if outlier_config['auto_clean'] else '否'}")
        print(f"• 多特征联合检测: {'启用' if outlier_config['multi_feature_detection'] else '禁用'}")
        print(f"• 异常值比例假设: {outlier_config['contamination']*100}%")
        print(f"• 最少投票数: {outlier_config['min_votes']}")
        print(f"• Z-Score阈值: {outlier_config['z_score_threshold']}")
        print(f"• IQR倍数: {outlier_config['iqr_multiplier']}")
        
        target_features = outlier_config.get('target_features', [])
        if target_features:
            print(f"• 重点检测特征: {', '.join(target_features)}")
        else:
            print(f"• 重点检测特征: 所有特征")
        
        print(f"• 保存异常值报告: {'是' if outlier_config['save_outlier_reports'] else '否'}")
        print(f"• 对比分析: {'是' if outlier_config['comparison_analysis'] else '否'}")
    
    # 创建主输出目录
    os.makedirs(CONFIG['pdp_output_dir'], exist_ok=True)
    
    # 分析GBDT模型
    print(f"\n" + "="*60)
    success = analyze_gbdt_model()
    
    if success:
        print(f"\n" + "="*80)
        print(f"GBDT模型PDP分析完成！")
        print(f"="*80)
        print(f"输出目录: {CONFIG['pdp_output_dir']}")
        
        print(f"\n详细结果保存在以下目录中:")
        print(f"  • combined_pdp/ (统一清洗PDP分析)")
        print(f"  • interaction_2d_pdp/ (统一清洗2D交互分析)")
        print(f"  • interaction_3d_pdp/ (统一清洗3D交互分析)")
        print(f"  • outlier_detection/ (全局异常值检测报告)")
        print(f"  • cleaned_analysis/ (清洗前后对比)")
        
        print(f"\n生成的统一异常值处理分析类型:")
        print(f"  • 统一清洗Individual PDP图：{CONFIG['grid_resolution']}网格点 + Excel数据")
        print(f"  • 统一清洗2D交互图：{CONFIG['interaction_2d_resolution']}×{CONFIG['interaction_2d_resolution']} 所有特征对 + Excel数据")
        print(f"  • 统一清洗3D交互图：{CONFIG['interaction_3d_resolution']}³ 重要特征组合 + 切片可视化 + Excel数据")
        print(f"  • 全局异常值检测报告：多种检测方法 + 多特征联合 + 投票机制 + Excel数据")
        print(f"  • 清洗前后对比：PDP曲线+分布+斜率+统计对比 + Excel数据")
        print(f"  • 交互强度排序：2D和3D交互的强度分析和排序（基于统一清洗数据）")
        
        print(f"\n统一异常值处理核心改进:")
        print(f"  • 全局统一异常值检测 (一次检测，全局应用)")
        print(f"  • 多特征联合异常值检测 (马氏距离+多维IsolationForest)")
        print(f"  • 智能投票机制 (多种检测方法综合决策)")
        print(f"  • 数据一致性保证 (所有PDP和交互分析使用相同的清洁数据)")
        print(f"  • 针对性Ω特征处理 (重点检测列表可配置)")
        print(f"  • PDP跳跃专项检测 (专门解决异常跳跃问题)")
        print(f"  • 完整可视化报告 (异常值分布+投票+清洗效果)")
        print(f"  • 清洗前后完整对比 (PDP曲线+分布+斜率+统计对比)")
        
        print(f"\n针对Ω特征异常跳跃问题的完整解决方案:")
        if 'Ω' in CONFIG['outlier_detection'].get('target_features', []) or not CONFIG['outlier_detection'].get('target_features', []):
            print(f"  • Ω特征已包含在统一异常值检测范围内")
            print(f"  • 6种单特征+2种多特征方法会综合识别Ω的异常值")
            print(f"  • PDP跳跃专项检测会找到导致跳跃的具体数据点")
            print(f"  • 统一清洗会在所有分析中移除异常数据点")
            print(f"  • 清洗前后对比会显示Ω特征的改进效果")
            print(f"  • 全局异常值报告会详细记录Ω特征的检测结果")
            print(f"  • 所有PDP和交互分析都将使用清洗后的Ω特征数据")
        else:
            print(f"  • ⚠️ Ω特征不在重点检测列表中")
            print(f"  • 建议将'Ω'添加到CONFIG['outlier_detection']['target_features']中")
        
    else:
        print(f"\nGBDT模型PDP分析失败")
    
    # 最终清理
    cleanup_matplotlib()
    import gc
    gc.collect()
    print(f"\n所有资源已清理，程序执行完毕")
    
    return success

if __name__ == "__main__":
    # 执行主程序
    success = main()


GBDT模型PDP分析程序（统一异常值处理版）
输出目录: D:\博一下\manuscript5-变Si组\GBDT模型PDP分析结果
分析类型: GBDT模型统一异常值处理 + 完整PDP和交互分析
matplotlib后端: Agg
交互模式: 关闭

统一异常值处理配置:
• 网格分辨率: 200点 (Individual PDP)
• 2D交互分辨率: 50×50 (所有特征对)
• 3D交互分辨率: 30³ (重要特征组合)
• 最大3D交互数: 10 (计算效率考虑)
• Bootstrap次数: 小数据集2000, 中等1000, 大型500
• 置信水平: 95.0%
• 直方图箱数: 50 (特征分布)

统一异常值检测配置:
• 异常值检测: 启用
• 统一清洗: 启用
• 自动清洗: 是
• 多特征联合检测: 启用
• 异常值比例假设: 5.0%
• 最少投票数: 2
• Z-Score阈值: 3.0
• IQR倍数: 1.5
• 重点检测特征: Ω
• 保存异常值报告: 是
• 对比分析: 是


[1/1] 分析GBDT模型: PCC_k=5_GradientBoosting
  特征: mean_C4 enthalpy melting, solubility_limit_factor, mean_S10 Lattice Constants a, Λ, Ω
=== 加载GBDT模型和数据 ===
✓ 成功加载GBDT模型: D:\博一下\manuscript5-变Si组\最优模型GBDT专用训练\optimal_GBDT_model_1756353666.pkl
✓ 成功加载模型信息: D:\博一下\manuscript5-变Si组\最优模型GBDT专用训练\model_info_1756353666.pkl
✓ 成功加载原始数据: (221, 6)
✓ 数据清理完成: (221, 5)
✓ 特征: ['mean_C4 enthalpy melting', 'solubility_limit_factor', 'mean_S10 Lattice Constants a', 'Λ', 'Ω']
✓ 目标变量: KQ
✓ 成功构造模型信息，准备进行PDP分析
  性能: R²=0.922975, MAE=2.218439
  数据集大小: 2